## github рендерит ipynb с ошибками, поэтому используйте nbviewer:
[https://nbviewer.org/github/StalSkyle/stars_classification/blob/main/UNFINISHED_multi_classification.ipynb?flush_cache=true](https://nbviewer.org/github/StalSkyle/stars_classification/blob/main/UNFINISHED_multi_classification.ipynb?flush_cache=true)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

# ссылка на данные
# https://drive.google.com/file/d/1wTkOoA222guACzvIJxLf4wU77Rykp1__/view?usp=sharing
df = pd.read_parquet("./data/B_vsx_vsx.parquet")
pd.set_option("display.max_columns", None)
plt.rcParams["figure.figsize"] = (16, 12)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9750415 entries, 0 to 9750414
Data columns (total 20 columns):
 #   Column    Dtype  
---  ------    -----  
 0   OID       int32  
 1   n_OID     object 
 2   Name      object 
 3   V         uint8  
 4   Type      object 
 5   l_max     object 
 6   max       float32
 7   u_max     object 
 8   n_max     object 
 9   f_min     object 
 10  l_min     object 
 11  min       float32
 12  u_min     object 
 13  n_min     object 
 14  l_Period  object 
 15  Period    float64
 16  u_Period  object 
 17  Sp        object 
 18  RAJ2000   float64
 19  DEJ2000   float64
dtypes: float32(2), float64(3), int32(1), object(13), uint8(1)
memory usage: 1.3+ GB


In [2]:
def classify_type(vtype: str) -> str:
    """
    Возвращает укрупнённый класс переменной звезды
    в зависимости от содержимого строки vtype.
    """

    if pd.isna(vtype) or not isinstance(vtype, str) or vtype.strip() == "":
        return "UNKNOWN"

    # Приведём к верхнему регистру для надёжного поиска подстрок
    t = vtype.upper()

    # --- 1) Затменные (Eclipsing Binaries) ---
    ecl_markers = ["EA", "EB", "EW", "EC", "ELL", "E/RS", "E|", "E "]
    if any(m in t for m in ecl_markers):
        return "ECLIPSING"

    # --- 2) Цефеиды и родственные (DCEP, CW, RV Tauri, ACEP) - пульсирующие
    cep_markers = ["DCEP", "CW-FU", "CW", "CWA", "CWB", "RVA", "RV", "ACEP", "CEP"]
    if any(m in t for m in cep_markers):
        return "PULSATING"

    # --- 3) RR Лиры (RRAB, RRC, RRD, RR...) - пульсирующие
    rr_markers = ["RRAB", "RRC", "RRD", "RR"]
    if any(m in t for m in rr_markers):
        return "PULSATING"

    # --- 4) Короткопериодические пульсаторы: DSCT, SXPHE, GDOR, roAp - пульсирующие
    short_puls = ["DSCT", "HADS", "SXPHE", "GDOR", "ROAP", "ROAM"]
    if any(m in t for m in short_puls):
        return "PULSATING"

    # --- 5) Долгопериодические и полуправильные (M, SR, L) - пульсирующие
    lpv_markers = ["MIRA", "SR", "SRA", "SRB", "SRC", "SRD", "L ", "LB", "LC", "LPV"]
    if any(m in t for m in lpv_markers):
        return "PULSATING"

    # --- 6) Ротационные переменные (BY, RS, ACV, SPB, ROT, GCAS) ---
    rot_markers = ["BY", "RS", "ACV", "SPB", "ROT", "GCAS"]
    if any(m in t for m in rot_markers):
        return "ROTATING"

    # --- 7) Эруптивные/молодые звёзды (T Tauri, EXOR, UXOR, INS...) ---
    yso_markers = ["TTS", "EXOR", "UXOR", "INS", "IN", "INST", "CST", "DYP", "FSCM", "FUOR", "YSO"]
    if any(m in t for m in yso_markers):
        return "ERUPTIVE"

    # --- 8) Катаклизмические (UG, NL, AM, ZAND, IB, IS, ... ) ---
    cataclysmic_markers = ["UG", "NL", "AM", "ZAND", "IB", "ISB", "BE", "DPV", "EXOR", "FUOR", "PNB"]  # и др.
    if any(m in t for m in cataclysmic_markers):
        return "ERUPTIVE"

    if (t == "E"):
        return "ECLIPSING"

    if (t == "L"):
        return "PULSATING"

    # если ничего не подошло
    return "UNKNOWN"

In [3]:
# целевая переменная:
df["class"] = df["Type"].apply(classify_type)

# в датасете очень мало эруптивных звезд
print(df["class"].value_counts())

class
ROTATING     3102285
PULSATING    3089412
ECLIPSING    2568649
UNKNOWN       892890
ERUPTIVE       97179
Name: count, dtype: int64


In [4]:
df = df[df["class"] != "UNKNOWN"]
df.drop("Type", inplace=True, axis=1)
df.head()

,OID,n_OID,Name,V,l_max,max,u_max,n_max,f_min,l_min,min,u_min,n_min,l_Period,Period,u_Period,Sp,RAJ2000,DEJ2000,class
0,8278100,,Gaia DR3 4685168858707787776,0,,16.59,,G,,,17.740000,,G,,NaN,,K,0.00006,-75.86906,PULSATING
1,2535232,,Gaia DR3 2881873169572728832,0,,16.59,,G,,,16.639999,,G,,2.35407,,G,0.00013,39.89248,ROTATING
2,2535233,,Gaia DR3 4918216945285915648,0,,17.07,,G,,,17.600000,,G,,NaN,,F,0.00019,-59.55921,PULSATING
3,2535234,,Gaia DR3 566749663745310848,0,,15.57,,G,,,15.620000,,G,,NaN,,K,0.00038,81.91627,ROTATING
4,2535235,,Gaia DR3 430093700005970048,0,,14.50,,G,,,14.530000,,G,,NaN,,F,0.00043,63.43314,ROTATING


In [5]:
df.isnull().sum()

OID               0
n_OID             0
Name              0
V                 0
l_max             0
max             443
u_max             0
n_max             0
f_min             0
l_min             0
min           13462
u_min             0
n_min             0
l_Period          0
Period      4593132
u_Period          0
Sp                0
RAJ2000           0
DEJ2000           0
class             0
dtype: int64

In [6]:
# несодержательные признаки
df.drop(["l_max", "u_max", "l_min", "u_min", "l_Period", "u_Period", "OID", "n_OID", "V"], inplace=True, axis=1)
# вряд ли нам понадобятся координаты
df.drop(["RAJ2000", "DEJ2000"], inplace=True, axis=1)

In [7]:
# попробуем извлечь новые признаки из колонки name
df["Name"] = df["Name"].str.split().str[0]

In [8]:
counts = df["Name"].value_counts()
common_types = counts[counts > 1000].index  # часто встречающиеся типы
df["Name"] = df["Name"].apply(lambda x: x if x in common_types else "err")

In [9]:
df["Name"].value_counts() # возможно, затем придётся что-то делать с err

Name
Gaia         6875067
err           856116
ZTF           649501
ASASSN-V      312531
KIC            34633
PS1-3PI        29261
WISE           22634
ASAS           18788
NSVS            8567
MACHO           5744
NSV             4969
OGLE            4078
LINEAR          3991
KID             3947
[CAG2000]       2905
HAT             2877
2MASS           2842
LMC             2578
CoRoT           2349
NGC             2271
EPIC            1953
SEKBO           1885
GSC             1844
HD              1734
BESTII          1414
SDSS            1019
[MAA2010]       1015
[H97b]          1012
Name: count, dtype: int64

In [10]:
# а пока что применим label encoder

le = LabelEncoder()
df["Name"] = le.fit_transform(df["Name"])

In [11]:
df.shape

(8857525, 9)

In [12]:
# теперь перед нами моральный выбор: дропнуть period, или дропнуть строки, в которых period nan
df[df["Period"].notna()]["class"].value_counts()

class
ECLIPSING    2509828
PULSATING    1017772
ROTATING      734431
ERUPTIVE        2362
Name: count, dtype: int64

У эруптивных звёзд *нет периода*, поэтому этот признак является очень важным для их определения.
Попробуем создать новый признак *has_period*$ у звёзд, у которого нет периода, зададим его значение как 0

In [13]:
df["has_period"] = df["Period"].apply(lambda x: 1 if pd.notna(x) else 0)
df["Period"] = df["Period"].fillna(0)

# поместим в df признак has_period после period
col = df.pop("has_period")
df.insert(8, "has_period", col)
df.head()

,Name,max,n_max,f_min,min,n_min,Period,Sp,has_period,class
0,7,16.59,G,,17.740000,G,0.00000,K,0,PULSATING
1,7,16.59,G,,16.639999,G,2.35407,G,1,ROTATING
2,7,17.07,G,,17.600000,G,0.00000,F,0,PULSATING
3,7,15.57,G,,15.620000,G,0.00000,K,0,ROTATING
4,7,14.50,G,,14.530000,G,0.00000,F,0,ROTATING


In [14]:
df["f_min"].value_counts()

f_min
     7165690
Y    1691835
Name: count, dtype: int64

In [15]:
df["f_min"] = df["f_min"].apply(lambda x: 1 if x == "Y" else 0)
df["f_min"].value_counts()

f_min
0    7165690
1    1691835
Name: count, dtype: int64

In [16]:
df.head()

,Name,max,n_max,f_min,min,n_min,Period,Sp,has_period,class
0,7,16.59,G,0,17.740000,G,0.00000,K,0,PULSATING
1,7,16.59,G,0,16.639999,G,2.35407,G,1,ROTATING
2,7,17.07,G,0,17.600000,G,0.00000,F,0,PULSATING
3,7,15.57,G,0,15.620000,G,0.00000,K,0,ROTATING
4,7,14.50,G,0,14.530000,G,0.00000,F,0,ROTATING


In [17]:
# всего 443 звезды не имеют признака max, поэтому дропнем их
df = df[df["max"].notna()]

# аналогично с min
df = df[df["min"].notna()]

In [18]:
counts = df["Sp"].value_counts()
common_types = counts[counts > 1000].index.to_list()
print(len(df[df["Sp"].isin(common_types)]), "нередкие с nan")
common_types.remove("")
print(len(df[df["Sp"].isin(common_types)]), "нередкие без nan")

8805306 нередкие с nan
4514800 нередкие без nan


In [19]:
# пусть редкий признак будет закодирован как "rare", а отсутствие признака - как "no"
df["Sp"] = df["Sp"].replace("", "none")
df["Sp"] = df["Sp"].apply(lambda x: "rare" if x not in common_types + ["none"] else x)

In [20]:
df["Sp"].value_counts()

Sp
none    4290506
K       1606290
M        871682
F        736688
G        711373
A        421088
B        120353
rare      38505
O         28113
C         15756
F0         1315
em         1123
G7         1019
Name: count, dtype: int64

In [21]:
df.head()

,Name,max,n_max,f_min,min,n_min,Period,Sp,has_period,class
0,7,16.59,G,0,17.740000,G,0.00000,K,0,PULSATING
1,7,16.59,G,0,16.639999,G,2.35407,G,1,ROTATING
2,7,17.07,G,0,17.600000,G,0.00000,F,0,PULSATING
3,7,15.57,G,0,15.620000,G,0.00000,K,0,ROTATING
4,7,14.50,G,0,14.530000,G,0.00000,F,0,ROTATING


In [30]:
# разберемся с n_max
# сделаем то же самое, что и с Sp
print(len(df[df["n_max"] == ""])) # всего 688 значений, можем смело дропать
df.shape

688


(8843811, 10)